In [10]:
# 📌 Step 1: Import All Required Libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split
from sklearn.metrics import classification_report

In [11]:
# 📌 Step 2: Load Dataset and Prepare Dataloaders

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

# Load Oxford-IIIT Pet dataset
data = datasets.OxfordIIITPet(
    root="./data",
    download=True,
    transform=transform,
    target_types="category"
)

# Split into train and validation sets
train_size = int(0.8 * len(data))
val_size = len(data) - train_size
train_data, val_data = random_split(data, [train_size, val_size])

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32)

100%|██████████| 792M/792M [00:20<00:00, 38.5MB/s]
100%|██████████| 19.2M/19.2M [00:01<00:00, 15.4MB/s]


In [12]:
# 📌 Step 3: Define the CNN Model

model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 37)  # 37 pet categories
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [13]:
# 📌 Step 4: Loss and Optimizer

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [14]:
# 📌 Step 5: Train the Model

for epoch in range(5):
    model.train()
    total_loss = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_loader):.4f}")

Epoch 1, Loss: 1.6507
Epoch 2, Loss: 0.7474
Epoch 3, Loss: 0.4740
Epoch 4, Loss: 0.3327
Epoch 5, Loss: 0.2414


In [15]:
# 📌 Step 6: Evaluate the Model

model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

print(classification_report(all_labels, all_preds))

              precision    recall  f1-score   support

           0       0.60      0.16      0.25        19
           1       0.50      0.35      0.41        20
           2       0.50      0.07      0.12        15
           3       0.58      0.62      0.60        24
           4       0.21      0.67      0.32        18
           5       1.00      0.18      0.31        22
           6       0.53      0.50      0.51        18
           7       0.57      0.57      0.57        14
           8       0.36      0.74      0.49        23
           9       0.29      0.29      0.29        24
          10       0.71      0.50      0.59        20
          11       0.50      0.47      0.48        15
          12       1.00      0.09      0.17        22
          13       0.00      0.00      0.00        20
          14       0.61      0.71      0.65        24
          15       0.78      0.37      0.50        19
          16       1.00      0.20      0.33        15
          17       0.67    

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
